# Análise e Controle de Sistemas Lineares Incertos por meio de Desigualdades Matriciais Lineares: Exercício

## Descrição

Eeste notebook explora conceitos avançados de análise e controle de sistemas lineares incertos através da formulação e resolução de problemas de programação linear utilizando as bibliotecas CVX e Mosek em Python, com ênfase em Inequações Matriciais Lineares (LMIs).

### Data de Criação

2024-01-21

### Data de Modificação

2024-01-22

## Autor

Andevaldo da Encarnação Vitório

## Importação das Bibliotecas Necessárias

In [141]:
import cvxpy as cp
import numpy as np
import math

## Problema 1

### Descrição

Considere o seguinte problema de otimização convexo baseado em desigualdades matriciais lineares:

$$ \underset{P}{\min} \space Tr(P) $$
$$ P > 0, \space\space A^TP+PA+Q < 0 $$

Resolva com

$$
A = \begin{bmatrix}
0 & 1 \\
-2 & -3
\end{bmatrix}
, \space
Q = 
\begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
$$


Resolva agora com o seguinte problema de otimização alternativo:

$$ \underset{P,\space \rho}{\min}\space Tr(\rho) $$
$$ P > 0, \space\space A^TP + PA + Q < 0, \space\space \rho \geq Tr(P) $$

In [142]:
# Problema 1 - Parte 2

# Definição da Variáveis P e rho
P = cp.Variable((2,2), symmetric=True)
rho = cp.Variable()

# Definiçaõ dos Parâmetros
A = cp.Parameter((2,2), value=[[0., -2.],[1., -3.]])
Q = cp.Parameter((2,2), value=np.eye(2))

# Apresentação dos parâmetros
# print('A =\n', A.value)
# print('\nQ =\n', Q.value)

# Definição do problema de otimização convexa: objetivo e restrições
obj = cp.Minimize(rho)
constraints = [P >> 0, A.T @ P + P @ A + Q << 0, rho >= cp.trace(P)]
prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("\nValor ótimo:", prob.value, '\n')

# Apresentação do resultado obtido na resolução
print("P = \n", P.value, '\n')
print("rho = \n", rho.value)


Valor ótimo: 1.4999999999962756 

P = 
 [[1.25 0.25]
 [0.25 0.25]] 

rho = 
 1.4999999999962756


## Problema 2

### Descrição

Para ilustrar o cômputo da norma $\mathscr{H}_2 = \lVert H(s) \rVert_2$ usando LMIs, considere o sistema massa-mola apresentado em (Iwasaki, 1996), ilustrado na figura abaixo:

<br>
<div style="text-align:center;">
  <picture>
    <source media="(prefers-color-scheme: light)" srcset="assets/imgs/sistema_massa_mola_light.png">
    <source media="(prefers-color-scheme: dark)" srcset="assets/imgs/sistema_massa_mola_dark.png">
  
  <img src="assets/imgs/sistema_massa_mola_dark.png" alt="Sistema Massa Mola" width="60%"   title="Sistema Massa Mola">
  </picture>
</div>
<br>

A mesma função de transferência de (Iwasaki, 1996) é considerada, isto é, da força de entrada $d$ aplicada à massa $m_1$ para o sinal de erro $e = x_2$ (posição da massa $m_2$).

As matrizes $A$, $B$ e $C$ são dadas por

<br>

$$
A = \begin{bmatrix}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\displaystyle-\frac{k_1 + k_2}{m_1} & \displaystyle\frac{k_2}{m_1} &\displaystyle-\frac{c_0}{m_1} & 0 \\
\displaystyle\frac{k_2}{m_2} &\displaystyle-\frac{k_2}{m_2} &0 &\displaystyle-\frac{c_0}{m_2}
\end{bmatrix}
, \space
B =
\begin{bmatrix}
0 \\
0 \\
\displaystyle \frac{1}{m_1} \\
0 \\
\end{bmatrix}
, \space
C =
\begin{bmatrix}
0 & 1 & 0 & 0
\end{bmatrix}
$$

<br>

Os valores dos parâmetros são: massas $m_1 = 1$ e $m_2 = 0.5$; elasticidade das molas $k_1 = k_2 = 1$ e as forças de atrito $f_1$ e $f_2$ dadas por $f_i = −c_0 \cdot \dot{x_i}, \space (i = 1, 2)$, sendo que
$c_0 = 2$ é o coeficiente de atrito viscoso.

Determine a norma $\mathscr{H}_2$ deste sistema usando o seguinte problema de otimização:

$$ \underset{\rho, \space W = W^T > 0}{\min} \space\space \rho $$

sujeito a

$$
rho \geq Tr(CWC^T), \space
AW + WA^T + BB^T < 0
$$

Na solução ótima $p = p^\star$, tem se

$$ \lVert H(s) \rVert ^ 2 \_ 2 = p^\star $$

In [143]:
# Problema 2

# Definição dos Parâmetros
m1 = 1.
m2 = 0.5
k1 = 1.
k2 = 1.
c0 = 2.

A = cp.Parameter((4, 4), value=[[0., 0., - ((k1 + k2) / m1), (k2 / m2)],
                               [0., 0., (k2 / m1), - (k2 / m2)],
                               [1., 0., - (c0 / m1), 0],
                               [0., 1., 0., -(c0 / m2)]])
B = cp.Parameter((4, 1), value=[[0., 0., 1. / m1, 0.]])
C = cp.Parameter((1, 4), value=[[0], [1], [0], [0]])

print('Parâmetros:\n')
print("A =\n", A.value, '\n')
print("B =\n", B.value, '\n')
print("C =\n", C.value, '\n')

# Definição das Variáveis
rho = cp.Variable()
W = cp.Variable((4, 4), symmetric=True)

# Definição do Problema: Objetivo e Restrições
obj = cp.Minimize(rho)
constraints = [W >> 0]
constraints += [rho >= cp.trace(C @ W @ C.T)]
constraints += [A @ W + W @ A.T + B @ B.T << 0]
prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("\nValor ótimo:", prob.value, '\n')

# Apresentação dos Resultados
print("W =\n", np.round(W.value, 3), '\n')
print("rho =\n", rho.value, '\n')
print("Norma H2 =\n", math.sqrt(rho.value), '\n')

Parâmetros:

A =
 [[ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [-2.  1. -2.  0.]
 [ 2. -2.  0. -4.]] 

B =
 [[0.]
 [0.]
 [1.]
 [0.]] 

C =
 [[0. 1. 0. 0.]] 


Valor ótimo: 0.0925925924451939 

W =
 [[ 0.157  0.083 -0.     0.037]
 [ 0.083  0.093 -0.037 -0.   ]
 [-0.    -0.037  0.231  0.   ]
 [ 0.037 -0.     0.     0.019]] 

rho =
 0.0925925924451939 

Norma H2 =
 0.30429030948289154 

